# Preparación de los datos del entrenamiento

&nbsp;

El <strong>tratamiento de los datos</strong>, previo al entrenamiento de la red, es una parte muy importante en este tipo de análisis. Las maneras de representar esa información son muy variadas y es necesario definir una representación que permita extraer la información fácilmente.



## 1. Estructuras de datos

En este primer apartado se describe como es, desde el punto de vista de la programación, la estructura en vectores que va a entrenar la red neuronal.

También se describe como son los datos que se dan para entrenar la red neuronal: el formato en el que vienen dados (fichero csv), las variables disponibles para describir los eventos... etc.

### 1.1 Formato de muestras para el entrenamiento de la red neuronal

La red se va a programar utilizando el entorno [Keras](https://keras.io/), una librería de Python que permite implementar algoritmos de Deep Learning de manera relativamente simple. Este entorno se describirá en el siguiente Notebook.

Keras está programado para trabajar con el paquete [NumPY](http://www.numpy.org/), que introduce en Python estructuras vectoriales e incluye gran variedad de métodos para trabajar con ellas. Los vectores que van a representar a los eventos van a definirse con este paquete. Estos vectores se declaran como objetos ```numpy.array``` y cada elemento que contienen se corresponde con un elemento del vector:

```
un_vector = numpy.array([elemento1, elemento2, ..., elementoN])
```

Trabajando con Keras, cada evento ( vector $\vec{x}^{(i)}$ ) se va a representar con un unico objeto ```numpy.array``` de tal manera que cada elemento del vector se corresponda con el valor que toma cada variable para ese evento:

```
evento = numpy.array([variable1, variable2, ..., variableN])
```

Por ejemplo, suponiendo que solo se contase con 3 variables por evento para entrenar la red: el momento transverso del primer lepton, el número de jets y la MET. Se cuenta con un evento donde esas variables toman valores de 33.3 GeV, 2 y 147.0 GeV respectivamente. Entonces ese evento podría representarse para ser leído por la red como:

```
evento = numpy.array([33.3, 2.0, 147.0])
```

En la práctica se verá que para que la red trabaje de manera más óptima hay ciertos tratamientos previos que se les pueden aplicar a los valores de las variables (como la normalización) para facilitar el proceso de aprendizaje. 
 



### 1.2 Raw Data

Los datos se dan en un fichero .csv con toda la información necesaria para el entrenamiento de la red neuronal. Exceptuando el heading, <strong>cada fila se corresponde con un evento de la muestra</strong>.

La estructura de vectores ```numpy.array``` descrita en el apartado 1.1 debe crearse a partir de la información guardada en este csv.

El fichero csv tiene un total de 33 columnas, que contienen toda la información para cada uno de los eventos disponibles. Tres de las columnas son información acerca del evento que no aportan información física acerca de la colisión, y por lo tanto no se utilizan para entrenar la red. Estas variables son:


-```EventId```: Número entero utilizado como número identificador del evento.

-```Weight```: Peso para calcular la significancia de señal una vez entrenada la red (que no se utiliza en esta práctica).

-```Label```: La etiqueta del evento, en formato ```string```, que indica si el evento es un evento de señal (```'s'```) o un evento de fondo (```'b'```). No se incluye en las muestras de los eventos que van a entrenar la red pero se utiliza para indicarle la clase a la que pertenece un evento y así ejecutar el entrenamiento.

Las otras 30 variables son variables físicas que describen la colisión y caracterizan el evento. Son las variables que entrenan la red neuronal:

-```DER_mass_MMC```

-```DER_mass_transverse_met_lep```

-```DER_mass_vis```

-```DER_pt_h```

-```DER_deltaeta_jet_jet```

-```DER_mass_jet_jet```

-```DER_prodeta_jet_jet```

-```DER_deltar_tau_jet```

-```DER_pt_tot```

-```DER_sum_pt```

-```DER_pt_ratio_lep_tau```

-```DER_met_phi_centrality```

-```DER_lep_eta_centrality```

-```PRI_tau_pt```

-```PRI_tau_eta```

-```PRI_tau_phi```

-```PRI_lep_pt```

-```PRI_lep_eta```

-```PRI_lep_phi```

-```PRI_met```

-```PRI_met_phi```

-```PRI_met_sumet```

-```PRI_jet_num```

-```PRI_jet_leading_pt```

-```PRI_jet_leading_eta```

-```PRI_jet_leading_phi```

-```PRI_jet_subleading_pt```

-```PRI_jet_subleading_eta```

-```PRI_jet_subleading_phi```

-```PRI_jet_all_pt```



En algunos eventos es posible que una o algunas de las variables no existan o no tengan significado físico i.e. la variable```PRI_jet_subleading_pt``` (el momento transverso del segundo jet más energético en la colisión) cuando hay menos de 2 jets (```PRI_jet_num``` < 2). En estos casos la variable toma un valor estándar de -999.0. 


Los eventos deben ser leídos por la red neuronal con el mismo formato (vectores). La red neuronal establece las correlaciones entre variables en función de la posición que ocupan en el vector y si se suprimen directamente los valores que no tienen significado entonces esa estructura desaparece. Por lo tanto, los valores perdidos no se pueden suprimir directamente del vector. Es decir, no es posible que la red lea un vector de 30 elementos y después uno de 29 porque hay uno que está perdido. En cambio, se le debe asignar a todos las entradas un valor único (como -999.0) que no figure en el rango del resto de variables que si están presentes y si tienen sentido físico. Se espera entonces, de esta manera, que la red aprenda por si sola a interpretar que ese valor no es relevante.


## 2. Lectura de datos en formato .csv

El paquete que se utiliza para leer los datos es [pandas](https://pandas.pydata.org/), una librería de Python con métodos para el análisis de datos.

Con este paquete se puede leer el csv <em>training.csv</em> y cargar la información en un objeto ```pandas.DataFrame``` ( variable ```data```) al que podemos acceder para ver la clase de información con la que estamos tratando.

Se miran las variables de los datos en el heading del csv y el número de eventos. De todas las variables del heading, se tienen que identificar las que van a servir para entrenar la red ( variable ```features```). Estas son todas menos ```'EventId'```, ```'Weight'``` y ```'Label'```. 

<p> Es muy importante saber cuantos eventos de cada clase hay en la muestra. En este caso, se tienen dos clases: </p>
<ul> 
    <li> <strong>Señal</strong>: Eventos con variable 'Label' = 's' </li>
    <li> <strong>Fondo</strong>: Eventos con variable 'Label' = 'b' </li>
</ul>    

In [1]:
import pandas

data = pandas.read_csv("training.csv", delimiter = ',') # DataFrame object with al the info of the csv

# Get the basic information:
print(">>> Heading:")
print(list(data)) # Features and other useful info
print("\n")

print(">>> Number of events: " + str(len(data)))
print("\n")

features = [h for h in list(data) if h not in ['EventId', 'Weight', 'Label']] # select the training features

print(features)
print(">>> Number of features: " + str(len(features)))
print("\n")

# Ocurrence of each class
n_signal = len(data.loc[data['Label'] == 's'])
n_background = len(data.loc[data['Label'] == 'b'])
print(">>> Number of signal events: " + str(n_signal))
print(">>> Number of background events: " + str(n_background))

>>> Heading:
['EventId', 'DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt', 'Weight', 'Label']


>>> Number of events: 250000


['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI

## 3. Representación de las variables

Se estudia la distribución de las variables de entrenamiento. Se importan los paquetes [NumPy](http://www.numpy.org/) para hacer uso de objetos ```numpy.array``` y [Matplotlib](https://matplotlib.org/) para generar los histogramas.

Se crea un histograma con la distribución de cada variable por separado para señal y fondo. Se excluyen de la representación los valores -999.0 para poder estudiar las distribuciones más fácilmente. Cada histograma se guarda con el nombre de la variable en la carpeta Histograms/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


### Loop over the features
for feature in features:
    
    print(">>> Plotting " + feature + " histogram...")
    
    # Signal and background values
    signal_values = list( data.loc[data['Label'] == 's', feature] )
    background_values = list( data.loc[data['Label'] == 'b', feature] )
    
    signal_values = list(filter(lambda x: x != -999.0, signal_values))
    background_values = list(filter(lambda x: x != -999.0, background_values))
    
    # Define the histogram binning
    xmin = min(signal_values + background_values) 
    xmax = max(signal_values + background_values)
    binning = np.linspace(xmin, xmax, 61) 
    
    # Plot and save the histogram
    plt.clf()
    plt.hist(signal_values, bins = binning, color = 'b', alpha = 0.3, histtype = 'stepfilled', 
             linewidth = 1, edgecolor = 'b', label = 'Signal')
    plt.hist(background_values, bins = binning, color = 'r', alpha = 0.3, histtype = 'stepfilled', 
             linewidth = 1, edgecolor = 'r', label = 'Background')
    plt.legend(loc = 'upper right')
    plt.xlabel(feature, fontsize = 12)
    plt.ylabel('Entries', fontsize = 12) 
    plt.savefig('Histograms/'+feature+'_histo.png', dpi = 600)
          
    print("    Done")

## 4. Creación de la estructura en vectores 

En este apartado se explica paso a paso como crear las muestras:

4.1. Estructura en vectores. Se introduce la estructura en vectores final a la cual se quiere llegar.

4.2. Normalización. Se explica como se deben normalizar los valores de las variables para que el entrenamiento sea más eficiente.

4.3. La estructura de las muestras. Se juntan los dos conceptos anteriores, estructura y normalización, para explicar como crear las muestras. Se crean dos estructuras, una para los eventos de fondo y otra para los eventos de señal.

4.4. Definición de los sets de entrenamiento: train set y test set. 

4.5. El balance de clases.

4.6. Construcción final de los sets.

4.7. Guardar las variables del entrenamiento.

### 4.1 Estructura en vectores

Anteriormente se introdujo la representación de las muestras de eventos como vectores. También, en el apartado 1.1 se explicó como implementar los vectores en Python como objetos ```numpy.array```. En este apartado se explica como se deben de estrutucturar los vectores de cara al entrenamiento.

Para leer los eventos, la red neuronal recibe como input una matriz $\vec{X}$ con todos los vectores. Después, ella los va leyendo dependiendo de como se haya configurado el proceso de entrenamiento (siguiente Notebook). Esta matriz tiene como filas los vectores construidos a partir de los eventos:

$$\vec{X} = 
\begin{pmatrix}
\vec{x}^{(1)} \\
\vec{x}^{(2)} \\
... \\
\vec{x}^{(\text{nEventos})}
\end{pmatrix}$$

Las columnas de esta matriz serán, por lo tanto, los valores de cada una de las variables $j$, cada columna equivale a una variable:

$$\vec{X} = 
\begin{pmatrix}
\vec{x}^{(1)}_1 & \vec{x}^{(1)}_2 & ... & \vec{x}^{(1)}_{\text{nVariables}} \\
\vec{x}^{(2)}_1 & \vec{x}^{(2)}_2 & ... & \vec{x}^{(2)}_{\text{nVariables}} \\
... & ... & ...& ... \\
\vec{x}^{(\text{nEventos})}_1 & \vec{x}^{(\text{nEventos})}_2 & ... &\vec{x}^{(\text{nEventos})}_{\text{nVariables}} \\
\end{pmatrix}$$ 


Para <strong>implementar una matriz con NumPy</strong> se define un objeto ```numpy.array``` cuyos elementos son a su vez objetos ```numpy.array```. Un ejemplo de creación de una matriz puede ser:

```
matriz = numpy.array([[1, 2, 3], [4, 5, 6]])
```
que crearía una matriz de elementos:
$$
\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
\end{pmatrix}$$

Para acceder a un elemento de una matriz en Python se deben especificar la fila ```a``` y la columna ```b``` mediante corchetes, en ese orden (y teniendo en cuenta que los índices empiezan a contar desde 0). Por ejemplo, si quisiesemos cambiar el elemento 5 de la matriz anterior por un 18, habría que hacer:

```
matriz[1][1] = 8
```

Tal y como se explica en el apartado 4.3 la creación de la matriz de muestras se va a hacer elemento a elemento utilizando la sintaxis anterior.

### 4.2 Normalización

Los valores de las variables conviene que se encuentre en el mismo rango, e idealmente tomando valores pequeños. Para ello, los datos se normalizan <strong> variable por variable </strong>. 

<p> En este caso, los datos se normalizan de tal manera que: </p>
<ul>
    <li> Su valor central es 0 </li>
    <li> Su desviación estándar es igual a 1 </li>
</ul>

<p> La normalización se compone de dos pasos: </p>
<ol>
    <li> Para centrar la muestra en 0, se cogen los valores de esa variable de todas las muestras (columna del csv) y se calcula la media. Dicha media se le resta a cada valor. </li>
    <li> Para hacer que la distribución de valores tenga valor 1, se calcula la desviación estándar de la columna de valores de la variable y se divide cada uno por dicha desviación estandar. </li>
</ol>

De acuerdo a la estructura en vectores que va a leer la red, normalizar la muestra variable a variable se correspondería a aplicarle a cada elemento:

$$ x'^{(i)}_j = \dfrac{x^{(i)}_j - \bar{x}_j}{\sigma (x_j)}$$

Donde $\bar{x}_j$ es la media de la columna de valores de la variable $j$: 
$$\bar{x}_j = \dfrac{1}{N}\sum_{i = 1}^{N} x^{(i)}_j$$

Y $\sigma (x_j)$ la desviación estándar de los valores de la columna $j$:

$$\sigma(x_j) = \sqrt{\dfrac{\sum_{i = 1}^{N} (x^{(i)}_j - \bar{x}_j)}{N-1}}$$

---
> <strong> IMPORTANTE: </strong> Puesto que los elementos de valor -999.0 son elementos que carecen de importancia o que no están disponibles para el evento en la muestra de datos, no se deben de incluir en la normalización de la muestra. Por lo tanto, se excluyen del calculo de la media y desviación estándar de los valores de cada variable. En lugar de aplicarles la normalización como al resto de variables, el valor de -999.0 se sustituye por otro más cercano al valor 0, como -5.0. De esta manera, en la muestra normalizada, todos estos elementos tienen el valor -5.0, y la red neuronal tendrá que aprender que ese valor carece de significado.

---

Los vectores del entrenamiento se van a construir directamente con los valores normalizados, para ahorrar tiempo. Los valores medios de cada variable y sus desviaciones estándar se calculan directamente de las columnas del objeto DataFrame excluyendo los valores -999.0:

In [ ]:
# Initialization:
means = {} # dict with the means of every feature
stds = {} # dict with the standard deviations of every feature

for feature in features:
    true_values = data.loc[data[feature] != -999.0, feature]
    means[feature] = true_values.values.mean()
    stds[feature] = true_values.values.std()
    

### 4.3 La estructura de las muestras

En este apartado se describe como es <strong>la implementación en el código para generar la matriz que va a entrenar la red</strong>. Para ahorrar tiempo (la estructura de vectores es grande) los elementos de la matriz se normalizan directamente.

La idea es generar dos matrices, una con las muestras de señal y otra con las muestras de fondo. De esta manera se podrá controlar con más facilidad que muestras se quieren utilizar para entrenar la red y cuales se quieren reservar para evaluar su rendimiento después. Por lo tanto, se va a tener una matriz con los eventos de señal de dimension $N_{\text{señal}}$ x 30 y otra con los eventos de fondo de dimension $N_{\text{fondo}}$ x 30.

#### IMPLEMENTACIÓN:

El primer paso es crear una matriz vacía definiendo el número de filas y el número de columnas:
```
matrix = np.zeros(shape = (n_filas, n_columnas))
```

Después hay que identificar los índices de las filas del objeto ```pandas.DataFrame``` llamado ```data``` con los eventos (señal o fondo) que se van a utilizar para llenar la matriz (el número de índices debe corresponderse con el número de filas de la matriz que se va a llenar):

```
indices = data.loc[data['Label'] == 'Label'].index.values
```

Una vez identificados se recorren esas filas del ```pandas.DataFrame``` (eventos seleccionados) y cada una de las variables con un bucle ```for``` doble. Se accede al valor:

```
valor = data.loc[evento][variable]
```

y si ese valor es distinto de -999.0 entonces se normaliza con los valores de la media y desviación estándar definidos previamente y se llena el elemento de la matriz:

```
matriz[fila][variable] = (valor - medias[variable])/stds[variable]
```

si se quiere se puede redefinir otro valor para el valor nulo, siempre y cuando sea siempre el mismo.

---
> <strong>Nota:</strong> Este paso recorre las 30 variables de las 250000 muestras, por lo que tarda cierto tiempo. Con la implementación de este código crear la estructura de vectores separados en clases lleva 1 hora y 15 minutos, aproximadamente.
---

In [ ]:
import time # to know how much time does the array creation takes

x_signal = np.zeros(shape = (n_signal, 30)) # empty signal array
x_background = np.zeros(shape = (n_background, 30)) # empty background array


start = time.time()

### Signal array construction

signal_idx = data.loc[data['Label'] == 's'].index.values
signal_progress = [int(i*float(len(signal_idx))) for i in np.linspace(0, 1, 101)] # list for printing progress

# Loop over the signal events:
for i,s in enumerate(signal_idx):
    for j,feature in enumerate(features):
        
        value = data.loc[s][feature]
        if (value != -999.0): x_signal[i][j] = (value - means[feature])/stds[feature]
        else: x_signal[i][j] = -5.0
            
    # Print progress:
    if (i in signal_progress):
        print("Signal: "+ str(i)+"/"+str(len(signal_idx)) +" completed")
            
### Background array construction

background_idx = data.loc[data['Label'] == 'b'].index.values
background_progress = [int(i*float(len(background_idx))) for i in np.linspace(0, 1, 101)] # list for printing progress

# Loop over the background events:
for i,s in enumerate(background_idx):
    for j,feature in enumerate(features):
        
        value = data.loc[s][feature]
        if (value != -999.0): x_background[i][j] = (value - means[feature])/stds[feature]
        else: x_background[i][j] = -5.0
            
    # Print progress:
    if (i in background_progress):
        print("Background: "+ str(i)+"/"+str(len(background_idx)) +" completed")
        
stop = time.time()
print("\n")
print(">>> Used time: " + str(stop - start))
        

### 4.4  Definición de los sets de entrenamiento: train set y test set

<p> Con los numpy arrays construidos en el paso anterior se quieren construir los numpy arrays del entrenamiento: </p>
<ul>
    <li> Un numpy array <strong>x_train</strong> con los numpy arrays que van a entrenar la red </li> 
    <li> Un numpy array <strong>y_train</strong> con un array para cada array de <strong>x_train</strong> que indice la clase: <em>np.array([0])</em> si es de fondo o <em>np.array([1])</em> si es de señal. </li>
</ul>

<p> Y también se tienen que reservar unos eventos para evaluar el performance de la red una vez entrenada: </p>
<ul>
    <li> Un numpy array <strong>x_test</strong> con los numpy arrays que van a ser clasificados por la red </li> 
    <li> Un numpy array <strong>y_test</strong> con un array para cada array de <strong>x_test</strong> que indice la clase: <em>np.array([0])</em> si es de fondo o <em>np.array([1])</em> si es de señal. </li>
</ul>

Primero se deciden los eventos <strong>de cada clase</strong> que van a componer los arrays finales <strong>x_train</strong> y <strong>x_test</strong>:

In [ ]:
print(x_signal)

n_test = 5000 # events of each class reserved for test

# train samples:
x_train_signal = x_signal[n_test:]
x_train_background = x_background[n_test:]

# test samples:
x_test_signal = x_signal[:n_test]
x_test_background = x_background[:n_test]

print("Number of signal events in training set: " + str(len(x_train_signal)))
print("Number of background events in training set: " + str(len(x_train_background)))

### 4.5 Balance de clases

<p>En este punto, se debe de tratar el problema del <strong>balance de las clases</strong>. Los dos sets de muestras, train y test set, tienen que estar equilibrados en cuanto al número de muestras de cada clase que contienen. Esto se hace para evitar dos problemas en la clasificación:</p>
<ul> 
    <li> Si el train set tiene muchas más muestras de una clase que de otra es muy posible que "aprenda" a clasificar mejor una de las dos clases. Por ejemplo, suponiendo que tenemos muchos eventos de fondo y pocos de señal. La red sabrá decir que un evento es de fondo, pero al leer un evento de señal no sabrá distinguirlo y probablemente lo clasifique como fondo también. Lo más probable es que acabe clasificando todas las muestras como fondo, independientemente de la forma que tengan.</li>
    <li> Si el test set tiene diferente número de muestras de una clase que de otra entonces nuestra manera de evaluar la red no está bien planteada. Si por ejemplo la red clasifica mejor las muestras de una clase que las de otra y además hay muchas más muestras de esa clase en el test set, nos dará un muy alto nivel de acierto sin ser verdad, o uno muy malo al contrario.</li>
</ul>

El balance de clases en el test set se resuelve inmediatamente sacando el mismo número de muestras de una clase y de otra de las muestras totales.

<p>En cambio las muestras que quedan en el train set pueden no estar balanceadas, como es el caso. Esto se puede abordar de varias maneras:</p>
<ul>
    <li> Pesando las clases a nivel de entrenamiento. Consiste en hacer que la red de más importancia al entrenamiento de una clase que a otra añadiendo un mayor coste de esa clase en la función de perdida.</li>
    <li> Pesando las muestras a nivel de entrenamiento. Exactamente igual que el pesado de clases pero muestra a muestra </li>
    <li> Duplicar algunas muestras del train set (de la clases menos abundante) hasta igualar el número, para reducir el efecto. </li>
</ul>

En este ejemplo, se recurre a la última opción y se clonan algunas muestras de señal para igualar al número de muestras de fondo:

In [ ]:
x_extra = np.zeros(shape = (n_background-n_signal, 30)) # empty array that will contain the clonned samples

sample_index = 0 # index of the sample that is clonned

for i in range(0, len(x_extra)):
    
    x_extra[i] = x_train_signal[sample_index]    
    sample_index += 1
    
    if (sample_index > len(x_train_signal) -1): sample_index = 0
        
# Add the clonned samples to the signal train set:        
x_train_signal = np.concatenate((x_train_signal, x_extra), axis = 0)

# Check if the classes are balanced know:
print("Number of signal events in training set: " + str(len(x_train_signal)))
print("Number of background events in training set: " + str(len(x_train_background)))



### 4.6 Construcción final de los train y test sets

Una vez las muestras estan balanceadas, se crean los arrays de las muestras, <strong>x_train</strong> y <strong>x_test</strong>, y de las etiquetas, <strong>y_train</strong> y <strong>y_test</strong>.

Una vez creados, los las muestras se mezclan de manera aleatoria.

In [ ]:
import random

### Creation of the label arrays:
y_train_signal = np.full(shape = (len(x_train_signal), 1), fill_value = 1)
y_test_signal = np.full(shape = (len(x_test_signal), 1), fill_value = 1)
y_train_background = np.full(shape = (len(x_train_background), 1), fill_value = 0)
y_test_background = np.full(shape = (len(x_test_background), 1), fill_value = 0)

### Creation of the final samples
x_train = np.concatenate((x_train_signal, x_train_background), axis = 0)
x_test = np.concatenate((x_test_signal, x_test_background), axis = 0)
y_train = np.concatenate((y_train_signal, y_train_background), axis = 0)
y_test = np.concatenate((y_test_signal, y_test_background), axis = 0)

### Random shuffle the samples
# Train and test indexes
idx_train = np.arange(y_train.shape[0])
idx_test = np.arange(y_test.shape[0])

# Shuffle the indexes
random.shuffle(idx_train)
random.shuffle(idx_test)

# Order the samples according to the new indexes distribution
y_train = y_train[idx_train]
x_train = x_train[idx_train]
y_test = y_test[idx_test]
x_test = x_test[idx_test]

x_train
y_train

### 4.7 Guardar las variables del entrenamiento

Como el proceso de crear las muestras es largo y lleva cierto tiempo, no es aconsejable crear las muestras cada vez que la red neuronal se entrena.

Los train y test sets generados se guardan en un fichero pickle que permitirá acceder a ellos más adelante:

In [ ]:
import pickle

with open('training_variables.p', 'wb') as file_:
        pickle.dump([x_train, y_train, x_test, y_test], file_)